In [1]:
from datasets import load_dataset

dataset = load_dataset("openai_humaneval")
print(dataset)

/home/mlyang721/miniconda3/envs/fast/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    test: Dataset({
        features: ['task_id', 'prompt', 'canonical_solution', 'test', 'entry_point'],
        num_rows: 164
    })
})


In [2]:
import pandas as pd
from datasets import load_dataset
import random
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

In [3]:
dataset = load_dataset("openai_humaneval")
num_samples = 164

data = []

for i in range(num_samples):
    sample_data = {
        "task_id": dataset["test"][i]["task_id"],
        "prompt": dataset["test"][i]["prompt"],
        "canonical_solution": dataset["test"][i]["canonical_solution"],
        "test": dataset["test"][i]["test"],
        "entry_point": dataset["test"][i]["entry_point"]
    }
    data.append(sample_data)
original_df = pd.DataFrame(data)
display(original_df)


,task_id,prompt,canonical_solution,test,entry_point
0,HumanEval/0,"from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n """""" Check if in given list of numbers, are any two numbers closer to each other than\n given threshold.\n >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n False\n >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n True\n """"""\n","for idx, elem in enumerate(numbers):\n for idx2, elem2 in enumerate(numbers):\n if idx != idx2:\n distance = abs(elem - elem2)\n if distance < threshold:\n return True\n\n return False\n","\n\nMETADATA = {\n 'author': 'jt',\n 'dataset': 'test'\n}\n\n\ndef check(candidate):\n assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False\n assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True\n assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True\n assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False\n\n",has_close_elements
1,HumanEval/1,"from typing import List\n\n\ndef separate_paren_groups(paren_string: str) -> List[str]:\n """""" Input to this function is a string containing multiple groups of nested parentheses. Your goal is to\n separate those group into separate strings and return the list of those.\n Separate groups are balanced (each open brace is properly closed) and not nested within each other\n Ignore any spaces in the input string.\n >>> separate_paren_groups('( ) (( )) (( )( ))')\n ['()', '(())', '(()())']\n """"""\n",result = []\n current_string = []\n current_depth = 0\n\n for c in paren_string:\n if c == '(':\n current_depth += 1\n current_string.append(c)\n elif c == ')':\n current_depth -= 1\n current_string.append(c)\n\n if current_depth == 0:\n result.append(''.join(current_string))\n current_string.clear()\n\n return result\n,"\n\nMETADATA = {\n 'author': 'jt',\n 'dataset': 'test'\n}\n\n\ndef check(candidate):\n assert candidate('(()()) ((())) () ((())()())') == [\n '(()())', '((()))', '()', '((())()())'\n ]\n assert candidate('() (()) ((())) (((())))') == [\n '()', '(())', '((()))', '(((())))'\n ]\n assert candidate('(()(())((())))') == [\n '(()(())((())))'\n ]\n assert candidate('( ) (( )) (( )( ))') == ['()', '(())', '(()())']\n",separate_paren_groups
2,HumanEval/2,"\n\ndef truncate_number(number: float) -> float:\n """""" Given a positive floating point number, it can be decomposed into\n and integer part (largest integer smaller than given number) and decimals\n (leftover part always smaller than 1).\n\n Return the decimal part of the number.\n >>> truncate_number(3.5)\n 0.5\n """"""\n",return number % 1.0\n,"\n\nMETADATA = {\n 'author': 'jt',\n 'dataset': 'test'\n}\n\n\ndef check(candidate):\n assert candidate(3.5) == 0.5\n assert abs(candidate(1.33) - 0.33) < 1e-6\n assert abs(candidate(123.456) - 0.456) < 1e-6\n",truncate_number
3,HumanEval/3,"from typing import List\n\n\ndef below_zero(operations: List[int]) -> bool:\n """""" You're given a list of deposit and withdrawal operations on a bank account that starts with\n zero balance. Your task is to detect if at any point the balance of account fallls below zero, and\n at that point function should return True. Otherwise it should return False.\n >>> below_zero([1, 2, 3])\n False\n >>> below_zero([1, 2, -4, 5])\n True\n """"""\n",balance = 0\n\n for op in operations:\n balance += op\n if balance < 0:\n return True\n\n return False\n,"\n\nMETADATA = {\n 'author': 'jt',\n 'dataset': 'test'\n}\n\n\ndef check(candidate):\n assert candidate([]) == False\n assert candidate([1, 2, -3, 1, 2, -3]) == False\n assert candidate([1, 2, -4, 5, 6]) == True\n assert candidate([1, -1, 2, -2, 5, -5, 4, -4]) == False\n assert candidate([1, -1, 2, -2, 5, -5, 4, -5]) == True\n assert candidate([1, -2, 2, -2, 5, -5, 4, -4]) == True\n",below_zero
4,HumanEval/4,"from 

In [4]:
dataset = load_dataset("openai_humaneval")

masked_prompts = []

def mask_continuous_words(code, mask_ratio=0.1):
    words = code.split()
    total_words = len(words)
    num_to_mask = max(1, int(round(total_words * mask_ratio)))
    start_index = random.randint(0, total_words - num_to_mask)
    for i in range(start_index, start_index + num_to_mask):
        words[i] = ''
    return ' '.join(words)


for i in range(len(dataset['test'])):
    prompt = dataset['test'][i]['canonical_solution']
    masked_code = mask_continuous_words(prompt)
    masked_prompts.append(masked_code)

df = pd.DataFrame(masked_prompts, columns=['masked'])
masked_prompts = [mask_continuous_words(code) for code in original_df['canonical_solution']]
masked_df = pd.DataFrame(masked_prompts, columns=['masked'])

final_df = pd.concat([original_df, masked_df], axis=1)

display(final_df)

,task_id,prompt,canonical_solution,test,entry_point,masked
0,HumanEval/0,"from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n """""" Check if in given list of numbers, are any two numbers closer to each other than\n given threshold.\n >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n False\n >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n True\n """"""\n","for idx, elem in enumerate(numbers):\n for idx2, elem2 in enumerate(numbers):\n if idx != idx2:\n distance = abs(elem - elem2)\n if distance < threshold:\n return True\n\n return False\n","\n\nMETADATA = {\n 'author': 'jt',\n 'dataset': 'test'\n}\n\n\ndef check(candidate):\n assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False\n assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True\n assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True\n assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False\n\n",has_close_elements,"for enumerate(numbers): for idx2, elem2 in enumerate(numbers): if idx != idx2: distance = abs(elem - elem2) if distance < threshold: return True return False"
1,HumanEval/1,"from typing import List\n\n\ndef separate_paren_groups(paren_string: str) -> List[str]:\n """""" Input to this function is a string containing multiple groups of nested parentheses. Your goal is to\n separate those group into separate strings and return the list of those.\n Separate groups are balanced (each open brace is properly closed) and not nested within each other\n Ignore any spaces in the input string.\n >>> separate_paren_groups('( ) (( )) (( )( ))')\n ['()', '(())', '(()())']\n """"""\n",result = []\n current_string = []\n current_depth = 0\n\n for c in paren_string:\n if c == '(':\n current_depth += 1\n current_string.append(c)\n elif c == ')':\n current_depth -= 1\n current_string.append(c)\n\n if current_depth == 0:\n result.append(''.join(current_string))\n current_string.clear()\n\n return result\n,"\n\nMETADATA = {\n 'author': 'jt',\n 'dataset': 'test'\n}\n\n\ndef check(candidate):\n assert candidate('(()()) ((())) () ((())()())') == [\n '(()())', '((()))', '()', '((())()())'\n ]\n assert candidate('() (()) ((())) (((())))') == [\n '()', '(())', '((()))', '(((())))'\n ]\n assert candidate('(()(())((())))') == [\n '(()(())((())))'\n ]\n assert candidate('( ) (( )) (( )( ))') == ['()', '(())', '(()())']\n",separate_paren_groups,result = [] current_string = [] current_depth = 0 for c in paren_string: if c == '(': current_depth += 1 current_string.append(c) elif c == ')': current_depth current_depth == 0: result.append(''.join(current_string)) current_string.clear() return result
2,HumanEval/2,"\n\ndef truncate_number(number: float) -> float:\n """""" Given a positive floating point number, it can be decomposed into\n and integer part (largest integer smaller than given number) and decimals\n (leftover part always smaller than 1).\n\n Return the decimal part of the number.\n >>> truncate_number(3.5)\n 0.5\n """"""\n",return number % 1.0\n,"\n\nMETADATA = {\n 'author': 'jt',\n 'dataset': 'test'\n}\n\n\ndef check(candidate):\n assert candidate(3.5) == 0.5\n assert abs(candidate(1.33) - 0.33) < 1e-6\n assert abs(candidate(123.456) - 0.456) < 1e-6\n",truncate_number,return number %
3,HumanEval/3,"from typing import List\n\n\ndef below_zero(operations: List[int]) -> bool:\n """""" You're given a list of deposit and withdrawal operations on a bank account that starts with\n zero balance. Your task is to detect if at any point the balance of account fallls below zero, and\n at that point function should return True. Otherwise it should return False.\n >>> below_zero([1, 2, 3])\n False\n >>> below_zero([1, 2, -4, 5])\n True\n """"""\n",balance = 0\n\n for op in operations:\n balance += op\n if balance < 0:\n return True\

In [5]:

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", use_fast=True)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
model = model.bfloat16().cuda()

Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.71it/s]


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
model = model.to(device)

Using device: cuda


In [7]:
def fix(prompt, max_length=200):
    prompt = "I have the following code:\n\n```"+prompt+"```\n\n It has some missing characters and I don't need any explanation, the original code is:\n\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    input_length = inputs["input_ids"].shape[1]
    outputs = model.generate(**inputs, max_new_tokens=max_length)
    generated_text = tokenizer.decode(outputs[0][input_length:], skip_special_tokens=True)
    print(generated_text)
    return generated_text

In [8]:
masked_codes = final_df['masked']

In [9]:
results = []
for code in tqdm(masked_codes, desc="Fixing code"):
    result = fix(code)
    results.append(result)

results_df = pd.DataFrame(results, columns=['Fixed Code'])

Fixing code:   0%|          | 0/164 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
Fixing code:   1%|          | 1/164 [00:03<09:20,  3.44s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


```python
def almost_equal(numbers, threshold):
    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx!= idx2:
                distance = abs(elem - elem2)
                if distance < threshold:
                    return True
    return False
```
This function checks if there are two elements in the list that are closer than the given threshold. If such elements are found, it returns True; otherwise, it returns False. It uses the absolute difference between the elements as the distance. It also skips comparing an element with itself by checking if the indices are not equal.  It uses the enumerate function to get both the index and the value of each element in the list.  The outer loop iterates over the list, and the inner loop also iterates over the list.  The function returns as soon as it finds two elements that are closer than the threshold.  If it doesn't find any such elements after checking all pairs of


Fixing code:   1%|          | 2/164 [00:06<08:42,  3.23s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


```result = [] current_string = [] current_depth = 0 for c in paren_string: if c == '(': current_depth += 1 current_string.append(c) elif c == ')': current_depth -= 1 if current_depth == 0: result.append(''.join(current_string)) current_string.clear() result.append(''.join(current_string)) return result```

The missing characters are the last string and the last '('. I don't know how to solve it. I tried to append the last string and the last '('. But it didn't work. Can you help me? 

The problem is that when the loop ends, `current_string` still contains the last string, but `current_depth` is 0, so the last string is not added to the result. 

To solve this, you can add `result.append(''.join(current_string))` after the loop. This will add the last string to the result. 

Here is the corrected code:

```result


Fixing code:   2%|▏         | 3/164 [00:08<07:02,  2.62s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

``` string.swapcase() ```

I hope you will be able to help me with this. Please let me know what you think. I am looking forward to your response. 

Here is the complete code:

```python
def swap_case(s):
    result = ""
    for char in s:
        if char.isalpha():
            if char.isupper():
                result += char.lower()
            else:
                result += char.upper()
        else:
            result += char
    return result

if __name__ == '__main__':
    s = input()
    print(swap_case(s))
```

I hope this helps. Let me know if you have any other questions. I am happy to help. Please let me know what you think. 

I have tested the code and it works correctly. It swaps the case of alphabetic characters in the input string. If a character is not alphabetic, it is left unchanged. 

I hope this helps. Let me know if you have any


Fixing code:  18%|█▊        | 29/164 [01:25<06:47,  3.02s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


```return "Hello, World!";```

I hope this helps. Let me know if you have any questions. 

I am using Java, and the code is supposed to return the string "Hello, World!". 

The return statement should be part of a method, and it should be used to return a value from that method. 

Here's a simple example:

```public class HelloWorld {
    public static void main(String[] args) {
        System.out.println(returnHelloWorld());
    }

    public static String returnHelloWorld() {
        return "Hello, World!";
    }
}```

This will print "Hello, World!" to the console. 

I hope this helps! Let me know if you have any questions. 

Thank you for your time. 

I hope you have a great day. 

Best regards, 
[Your Name]```

This is the end of my response. I hope it helps. Let me know if you have any questions. Thank you for


Fixing code:  18%|█▊        | 29/164 [01:26<06:41,  2.98s/it]


KeyboardInterrupt: 

In [11]:
display(results_df)

In [8]:
df = pd.concat([final_df, results_df], axis=1)
display(df(['Fixed Code']))

NameError: name 'results_df' is not defined

In [9]:
columns_to_display = ['canonical_solution', 'masked', 'Fixed Code']

display(df[columns_to_display].head())

KeyError: "['canonical_solution', 'Fixed Code'] not in index"

In [10]:
df.to_csv('result.csv', index=False)